# Import Libraries

In [ ]:
import os
import numpy as np
import pydicom

import numpy as np
import matplotlib.pyplot as plt
from skimage import img_as_float, exposure
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio
from skimage.util import random_noise
import os


import pydicom
import skimage
print(skimage.__version__)

import cv2

# Add Gaussian Noise

https://medium.com/@ms_somanna/guide-to-adding-noise-to-your-data-using-python-and-numpy-c8be815df524

In [ ]:
def add_gaussian_noise(image, mean, std):
    """
    Add Gaussian noise to a 2D image.
    
    Parameters:
    - image: numpy array, the original image.
    - mean: mean of the Gaussian noise.
    - std: standard deviation of the Gaussian noise.
    
    Returns:
    - noisy_image: numpy array, the noisy image.
    """

    mean = 0 
    std = 0.21 * np.max(image)
    noise = np.random.normal(mean, std, image.shape)
    noisy_image = image + noise
    noisy_image = np.clip(noisy_image, 0, np.max(image))
    return noisy_image


# Image Directory

In [ ]:
def process_ct_images(input_dir, output_dir, mean, std):
    """
    Process all DICOM files in the input directory and its subdirectories, 
    add Gaussian noise, and save the noisy images to the output directory.

    Parameters:
    - input_dir: path to the directory with original CT-Scan images (can include subdirectories).
    - output_dir: path to save noisy images.
    - mean: mean of the Gaussian noise.
    - std: standard deviation of the Gaussian noise.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for root, _, files in os.walk(input_dir):
        for filename in files:
            file_path = os.path.join(root, filename)
            if os.path.isfile(file_path):
                try:
                    dicom = pydicom.dcmread(file_path)
                    image = dicom.pixel_array
                    
                    # Add Gaussian noise
                    noisy_image = add_gaussian_noise(image, mean, std)
                    
                    relative_path = os.path.relpath(root, input_dir)
                    sub_output_dir = os.path.join(output_dir, relative_path)
                    if not os.path.exists(sub_output_dir):
                        os.makedirs(sub_output_dir)
                    output_path = os.path.join(sub_output_dir, filename + ".dcm" if not filename.endswith(".dcm") else filename)
                    
                    noisy_dicom = dicom
                    noisy_dicom.PixelData = noisy_image.astype(np.uint16).tobytes()
                    noisy_dicom.save_as(output_path)
                    print(f"Processed: {file_path} -> {output_path}")
                except Exception as e:
                    print(f"Skipping {file_path}: Not a valid DICOM file ({e})")


 # Execute

In [ ]:
# Directories
input_directory = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Data Rapi/Original Angio/CATHLAB"
output_directory = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Data Rapi/Added Noise Angio/CATHLAB Noisy (0.21)"

# Add Gaussian noise
process_ct_images(input_directory, output_directory, mean=0, std=0.21)